In [ ]:
# Big Data Computing
## Specialisatie Minor Advanced Datamining & High Performance Computing
### Martijn Herber 2020

# Introductie
* Dit vak leert je grote datasets verdelen over meerdere CPU's, computers en GPU's
* Het draait er vooral om programmeren in python en het begrijpen van datasets
* Het is een praktisch vak met praktijkopdrachten
* Als je alle opdrachten hebt gemaakt en deze voldoen aan de eisen, heb je het vak gehaald.
* Er is een competitie-element toegevoegd: wie de snelste oplossingen schrijft, wint een prijs

## 3 paradigma's
* Duur woord voor "aanpak" :-)
* Hoeveel werk wil je zelf doen?
* "Manual" -> zelf alle data opsplitsen en verdelen
* "Library" -> de data wordt volgens een bepaald stramien voor je opgedeeld
* "GPU" -> er is niet zoveel wat je kunt berekenen, maar wat wel kan kan niet sneller


## Parallel processing, Distributed computing
* Moderne processoren (4-8 cores in een PC)
=> Parallel computing
* Meerdere computers op een netwerk
=> Parallel computing
* Duizenden computers verspreid in datacentra
=> Distributed processing

## Process-based distributed computing
* Processen de vorm waarin een OS programma's onafhankelijk draait
* Geen toegang tot elkaars geheugenruimte!
* Vanwege technische limitaties in Python is alleen process-based parallelisatie mogelijk
(Global Interpreeter Lock)

In [11]:
import multiprocessing as mp
from time import sleep
from random import randint

def todo(p):
    sleeptime = randint(1,5)
    sleep(sleeptime)
    print("hoi! van process",p , "lekker", sleeptime, "seconden geslapen.")

processes = []

for p in range(5):
    temP = mp.Process(target=todo, args=(p,))
    processes.append(temP)
    temP.start()
  # <-  

for p in processes:
    p.join()    
    
# <- alles is klaar!


hoi! van process 1 lekker 1 seconden geslapen.
hoi! van process 2 lekker 3 seconden geslapen.
hoi! van process 3 lekker 3 seconden geslapen.
hoi! van process 0 lekker 4 seconden geslapen.
hoi! van process 4 lekker 4 seconden geslapen.


In [12]:
import multiprocessing as mp
from time import sleep
from random import randint

def todo(p, outputQ):
    sleeptime = randint(1,5)
    sleep(sleeptime)
    outputQ.put("hoi! van process %s heeft %s sec geslapen" % (p , sleeptime))

processes = []
outputQ = mp.Queue()

for p in range(5):
    temP = mp.Process(target=todo, args=(p,outputQ))
    processes.append(temP)
    temP.start()

for p in processes:
    p.join()
    
while not outputQ.empty():
    msg = outputQ.get()
    print(msg)

hoi! van process 1 heeft 2 sec geslapen
hoi! van process 2 heeft 2 sec geslapen
hoi! van process 3 heeft 2 sec geslapen
hoi! van process 0 heeft 5 sec geslapen
hoi! van process 4 heeft 5 sec geslapen


## De Opdracht
### FastQ file verwerken
### "Embarassingly parallel" -> miljoenen dezelfde stukjes data
### Quality string is de 4e regel in een fastq bestand
### Geeft per base een kans weer dat de base klopt

Phred Quality Score:
\begin{equation}
Q = -10 log_{10} P
\end{equation}

Elke letter in regel 4 is een code voor getalwaarde van de phred score Q!
Deze is van ASCII om te zetten in een getal met de standaard python "ord()" functie.

In [2]:
#Voorbeeld fastq read:

#@SRR001666.2 071112_SLXA-EAS1_s_7:5:1:801:338 length=72
#GTTCAGGGATACGACGTTTGTATTTTAAGAATCTGAAGCAGAAGTCGATGATAATACGCGTCGTTTTATCAT
#+SRR001666.2 071112_SLXA-EAS1_s_7:5:1:801:338 length=72
#IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII6IBIIIIIIIIIIIIIIIIIIIIIIIGII>IIIII-I)8I

quals = "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII6IBIIIIIIIIIIIIIIIIIIIIIIIGII>IIIII-I)8I"

print([ord(c) for c in quals])

[73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 54, 73, 66, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 71, 73, 73, 62, 73, 73, 73, 73, 73, 45, 73, 41, 56, 73]


In [ ]:
# Om een idee te krijgen van de waarden en wat ze betekenen:
# (Laatste colom geeft de kans dat de base correct is)

10	1 in 10	90%
20	1 in 100	99%
30	1 in 1000	99.9%
40	1 in 10,000	99.99%
50	1 in 100,000	99.999%
60	1 in 1,000,000	99.9999%